# Metadata Extractor LLM Service

This notebook sets up a GPU-accelerated LLM service that extracts metadata from code repositories. The service uses:
- A fine-tuned Llama model running on Ollama
- Specialized system prompt for research metadata extraction
- Flask API exposed with ngrok for external access

### 1. System Dependencies

Install Linux system utilities

In [ ]:
!sudo apt-get update
!sudo apt-get install -y pciutils lshw

### 2. Ollama Installation

Install Ollama, which is the serving framework for running the LLM locally.

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

### 3. Hugging Face Integration

Install the Hugging Face hub library to download models.

In [ ]:
!pip install huggingface_hub

### 4. Download Pre-Trained Model

Download the fine-tuned Llama model in GGUF format from Hugging Face Hub.

In [ ]:
from huggingface_hub import hf_hub_download

model_name = "smutyala/llama_f16_gguf_model"
model_file = "unsloth.F16.gguf"

model_path = hf_hub_download(
    model_name,
    filename=model_file,
    local_dir='models/unsloth/',
    # If needed, Replace with your own token (Huggingface Hub Setting -> Access Tokens -> New token -> Generate Token)
    # token="YOUR_HUGGINGFACE_TOKEN_HERE"  
)

print("My model path:", model_path)

### 6. Model Configuration with Modelfile

Create a Modelfile to define the model parameters and system prompt for metadata extraction.

In [ ]:
modelfile_content = """
FROM ./unsloth.F16.gguf

PARAMETER num_gpu 999
PARAMETER num_ctx 50000
PARAMETER num_predict 3000

SYSTEM \"\"\"You are an advanced, award-winning metadata extraction system specializing in software-related metadata. Your unparalleled expertise enables you to accurately analyze and extract information with exceptional precision. You possess a deep understanding of programming languages, frameworks, dependencies, licenses, installation processes, authorship, funding sources, DOI identifiers, and all technical aspects related to software development.

REQUIRED FIELDS — ALL MUST BE LISTS OF STRINGS, EVEN IF EMPTY:
1. "Dependencies": Exact package names + versions.
2. "Installation_Instructions": Step-by-step or referenced filenames.
3. "Authors": Author names or extended details (as strings).
4. "Contributors": Names and contributions (as strings).
5. "Funding": Grant numbers, agencies.
6. "DOI": Any DOI identifiers.
7. "License": Name, version, and terms.
8. "Keywords": Mandatory — at least 5 technical terms describing the software domain, purpose, or architecture.

STRICT RULES:
- All values must be lists of strings.
- If any field has no valid data, return it as an empty list: [].
- Format the JSON as a single compact line with no indentation or extra spaces.
- Always use double quotes around keys and string values.
- DO NOT fabricate or guess.
- DO NOT use the examples below in real output.

EXAMPLE OUTPUT (REAL DATA PRESENT — DO NOT COPY):

{
"Dependencies": ["torch: >=2.1.0", "numpy", "scikit-learn: ^1.3"],
"Installation_Instructions": ["pip install -r requirements.txt", "Follow install.md"],
"Authors": ["Jane Doe (Lead Developer)", "John Smith (Researcher)"],
"Contributors": ["Alice Lee (Documentation)", "Tom Ray (Maintainer)"],
"Funding": ["EU Horizon Grant No. 123456", "NSF 987654321"],
"DOI": ["10.5281/zenodo.1234567", "10.1000/j.jss.2024.01.001"],
"License": ["MIT License"],
"Keywords": ["deep learning", "image classification", "PyTorch", "model training", "software toolkit"]
}

EXAMPLE OUTPUT (NO DATA FOUND — EMPTY STRUCTURE):

{
"Dependencies": [],
"Installation_Instructions": [],
"Authors": [],
"Contributors": [],
"Funding": [],
"DOI": [],
"License": [],
"Keywords": []
}

NEVER use the example values above in real outputs. They are only to show structure. Only extract from the provided input text.
\"\"\"

TEMPLATE \"\"\"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Extract metadata from the given input. Use the following fields: Dependencies, Installation_Instructions, Authors, Contributors, Funding, DOI, License, Keywords. Every field must be a list of strings — even if empty. Keywords must always include at least 5 relevant technical terms. Format your output as a single-line JSON string. Never use sample data — extract only from input.

### Input:
{{ .Prompt }}

### Response:
\"\"\"
"""

with open("Modelfile", "w") as f:
    f.write(modelfile_content.strip())

print("✅ Modelfile written successfully.")

### 7. Ollama Server Setup

Start the Ollama server with configured environment variables for optimal performance.

In [ ]:
import subprocess
import threading
import time
import os

def start_ollama():
    env = os.environ.copy()
    env["CUDA_VISIBLE_DEVICES"] = "0,1"
    env["OLLAMA_KEEP_ALIVE"] = "1h"
    subprocess.Popen(["ollama", "serve"], env=env)

thread = threading.Thread(target=start_ollama)
thread.start()
time.sleep(10)
print("Ollama server started and configured with OLLAMA_NUM_PARALLEL.")

### 7.1 Navigating to Model Directory

Change to the model directory to create the metadata extractor model.

In [ ]:
cd models/unsloth

### 7.2 Create Metadata Extractor Model

Build the metadata-extractor model using the Modelfile and downloaded weights.

In [ ]:
!ollama create metadata-extractor -f ./Modelfile

### 8. Model Testing

Test the model with a sample repository README to verify metadata extraction capabilities.

In [ ]:
import requests
import json

ollama_url = "http://localhost:11434/api/generate"

payload = {
    "model": "metadata-extractor",
    "prompt":  "# fsbrain [![DOI](https://zenodo.org/badge/209085379.svg)](https://zenodo.org/doi/10.5281/zenodo.3559816) An R package for structural neuroimaging. Provides high-level functions to access (read and write) and visualize surface-based brain morphometry data (e.g. cortical thickness) for individual subjects and groups. ![Vis](https://github.com/dfsp-spirit/fsbrain_gallery/blob/master/surface/fsbrain_sulcal_depth_cbar_web.jpg?raw=true \"Sulcal depth visualization, created with fsbrain\") **Fig.1**: *Visualization of sulcal depth for a subject in FreeSurfer standard space (fsaverage). See the [source code to reproduce this image](https://htmlpreview.github.io/?https://github.com/dfsp-spirit/fsbrain/blob/develop/web/Rmd_web_examples/examples_export.html) in an R notebook.* [About](#about) | [Installation](#installation) | [Documentation](#documentation) | [Unit tests](#unit-tests-and-continuous-integration) | [License](#license) | [Citation](#citation) | [Visualization examples](#visualization-examples) | [Contributing](#contributing) ## About The *fsbrain* R package provides a well-tested and consistent interface to neuroimaging data in [R](https://www.r-project.org/). It supports reading, writing, and visualizing various kinds of raw data and statistical results on brain surfaces and volumes. While the package provides a very convenient interface for working with data arranged in the standard [FreeSurfer](http://freesurfer.net/) directory structure (SUBJECTS_DIR), *fsbrain* is not limited to this layout or FreeSurfer file formats. You can load brain meshes, volumes, and data from a range of other neuroimaging software packages and visualize them. The plots produced by *fsbrain* can be integrated into R notebooks or written to high-quality bitmap image files, ready for publication. The [rgl](https://CRAN.R-project.org/package=rgl) renderer used by *fsbrain* provides fast, hardware-accelerated rendering based on the OpenGL standard. ## News * 2023-06-26: New fsbrain version 0.5.4 released on CRAN, see the [CHANGES](./CHANGES). * 2022-12-22: We are looking for help! If you have a Mac and are interested in helping fsbrain development, please [contact us by email](http://rcmd.org/ts/#contact) or reply to [#46](https://github.com/dfsp-spirit/fsbrain/issues/46) here on GitHub! * 2022-02-13: New fsbrain version 0.5.3 released on CRAN, see the [CHANGES](./CHANGES). * 2021-11-11: New fsbrain version 0.5.1 released on CRAN, see the [CHANGES](./CHANGES). * 2021-09-16: New fsbrain version 0.5.0 released on CRAN, see the [CHANGES](./CHANGES). * 2021-05-12: New fsbrain version 0.4.3 released on CRAN, see the [CHANGES](./CHANGES). * 2021-03-28: New fsbrain version 0.4.2 released on CRAN, see the [CHANGES](./CHANGES). * 2020-09-20: The preprint of our paper [T. Schaefer, C. Ecker: fsbrain: an R package for the visualization of structural neuroimaging data](https://doi.org/10.1101/2020.09.18.302935)' is now available on biorxiv. ## Installation ### Recommended: install the stable fsbrain version from CRAN You can find the [fsbrain package on CRAN](https://cran.r-project.org/package=fsbrain), so all you need to do is: ```r install.packages(\"fsbrain\"); ``` In case something goes wrong, don't worry. Just install the missing [system dependencies](#system-dependencies) and retry. ### Risky: install the dev version of fsbrain with the latest features This version is not guaranteed to be in a usable state, try at your own risk and run the tests before using it. From an R session: ```r install.packages(c(\"devtools\", \"knitr\", \"markdown\", \"rmarkdown\", \"testthat\", \"qpdf\")); devtools::install_github(\"dfsp-spirit/fsbrain\", build_vignettes=TRUE); ``` ### System dependencies A *system dependency* is a **non-R** software that is needed for the installation of a package. System dependencies cannot be installed automatically using the R package system, so you need to install them manually or using the package manager of your operating system. The *fsbrain* package itself does not have any system dependencies, however, it uses *rgl* for rendering. You can check the *SystemRequirements* section on the [rgl page at CRAN](https://CRAN.R-project.org/package=rgl) for the full list of rgl dependencies or read on. To get GIFTI format support, you will also need `libxml2-dev`. To install the system dependencies for *rgl* and *xml2*: #### Linux System dependencies (or: building from source) R packages are compiled from source by default under Linux, so you need some development libraries. Before installing *fsbrain*, run the following command in your system shell (not in R): * for deb-based Linux distributions (Debian, Ubuntu, ...): ```shell sudo apt-get install libmagick++-dev libx11-dev libgl1-mesa-dev libglu1-mesa-dev mesa-common-dev libfreetype6-dev libxml2-dev libssh-dev libcurl4-openssl-dev gfortran libblas-dev liblapack-dev libgfortran4 ``` Note: For recent Ubuntu versions, you may have to replace ```libgfortan4``` in the command above with ```libgfortan5```. * for rpm-based Linux distributions (Fedora, CentOS, RHEL, ...): ```shell sudo yum install ImageMagick-c++-devel libX11-devel mesa-libGLU-devel freetype-devel libxml2-devel ``` If you want to compile the package under any other operating system, you will need the libraries as well, of course. #### MacOS System dependencies Recent MacOS versions do not ship with an X11 environment. You will have to install the [xquartz X11 system](https://www.xquartz.org/) if you do not have it already. If you want to create GIF movies, make sure you have imagemagick installed (easiest via [homebrew](https://brew.sh/): `brew install imagemagick@6`). #### Windows Installation Hints Under Windows 10, it seems that you will need to install these two packages manually via the `install.packages` command: `shiny` and `manipulateWidget`. ### Installation via Docker There are Docker images for fsbrain available on Dockerhub, see the [fsbrain Dockerhub repo](https://hub.docker.com/r/dfspspirit/fsbrain). ## Documentation The documentation can be accessed from within an R session after you have loaded the *fsbrain* package: * There are two online R Markdown notebooks (like Jupyter Notebook in Python) that show various example plots in combination with the code used to produce them: * [basic fsbrain example notebook](https://htmlpreview.github.io/?https://github.com/dfsp-spirit/fsbrain/blob/develop/web/Rmd_web_examples/examples.html): Live visualization of subject data * [advanced fsbrain example notebook](https://htmlpreview.github.io/?https://github.com/dfsp-spirit/fsbrain/blob/develop/web/Rmd_web_examples/examples_adv.html): Plotting group data * [export API fsbrain example notebook](https://htmlpreview.github.io/?https://github.com/dfsp-spirit/fsbrain/blob/develop/web/Rmd_web_examples/examples_export.html): Exporting publication-ready plots * Detailed vignettes with explanations and examples for the functions of the package is included, run `browseVignettes(\"fsbrain\")` to see the vignettes. You can also open the vignette directly: * How to load and visualize surface-based neuroimaging data: `vignette(\"fsbrain\")` or: [read online at CRAN](https://cran.r-project.org/web/packages/fsbrain/vignettes/fsbrain.html) * How to load and visualize volume-based neuroimaging data: `vignette(\"fsbrain_vol\")` or: [read online at CRAN](https://cran.r-project.org/web/packages/fsbrain/vignettes/fsbrain_vol.html) * The fsbrain FAQ: `vignette(\"fsbrain_faq\")` or: [read online at CRAN](https://cran.r-project.org/web/packages/fsbrain/vignettes/fsbrain_faq.html) * Help for a specific function can be accessed in the usual R manner: `?<function>`, where you replace `<function>` with a function name. Like this: `?group.morph.native`. * Run `example(<function>)` to see a live demo that uses the function `<function>`. Like this: `example(group.morph.native)`. * The [unit tests](./tests/testthat/) that come with this package are essentially a list of examples that illustrate how to use the functions. ## Unit tests and Continuous integration This package comes with [lots of unit tests](./tests/testthat/). To run them, in a clean R session: ```r library(devtools) library(fsbrain) devtools::check() ``` Continuous integration results: <!-- badges: start --> [![AppVeyor build status](https://ci.appveyor.com/api/projects/status/github/dfsp-spirit/fsbrain?branch=master&svg=true)](https://ci.appveyor.com/project/dfsp-spirit/fsbrain) AppVeyor CI under Windows <!-- [![R-CMD-check](https://github.com/dfsp-spirit/fsbrain/workflows/R-CMD-check/badge.svg)](https://github.com/dfsp-spirit/fsbrain/actions) --> [GitHub Actions, Ubuntu Linux and MacOS](https://github.com/dfsp-spirit/fsbrain/actions) (Note: Currently this is always \"failing\" because of a warning caused by the `rgl` package when running headless. So the simlpe \"passing\"/\"failing\" status is useless, and one needs to follow the link to check the relevant CI results in detail.) <!-- badges: end --> ## License The *fsbrain* package is [free software](https://en.wikipedia.org/wiki/Free_software), published under the [MIT license](https://opensource.org/licenses/MIT). Note: The file LICENSE in this repository is a CRAN license template only (as required by CRAN) and does not contain the full MIT license text. See the file [LICENSE_FULL](./LICENSE_FULL) for the full license text. ## Citation and Publications You can generate the citation for [our fsbrain paper](https://doi.org/10.1101/2020.09.18.302935) by typing the following command in R: ``` citation(\"fsbrain\") ``` This currently outputs: ``` To cite fsbrain in publications use: Tim Schaefer, Christine Ecker (2020). fsbrain: an R package for the visualization of structural neuroimaging data. bioRxiv doi: 10.1101/2020.09.18.302935 A BibTeX entry for LaTeX users is @Misc{, title = {fsbrain: an {R} package for the visualization of structural neuroimaging data}, author = {Tim Schaefer and Christine Ecker}, year = {2020}, url = {https://www.biorxiv.org/content/10.1101/2020.09.18.302935v1}, doi = {10.1101/2020.09.18.302935}, } ``` Other materials related to fsbrain: * A poster on *fsbrain* has been presented at INSAR 2020 Annual Meeting: [Abstract](https://insar.confex.com/insar/2020/meetingapp.cgi/Paper/33181), [ePoster viewer](https://insar.confex.com/insar/2020/techdemo/eposter.cgi?eposterid=227), [PDF download](https://github.com/dfsp-spirit/fsbrain_gallery/raw/master/extra_materials/Poster_IMFAR2020_fsbrain.pdf) ## Visualization examples The *fsbrain* package support visualizations of different data, and all data can be displayed in one or more views. The figure below shows some examples for surface-based data: ![Visoverview](./web/fsbrain_vis_overview.jpg?raw=true \"Some visualization options from fsbrain\") **Fig.2**: *Example output for the fsbrain interactive visualization functions*. * **Subfigure A** shows the visualization of raw morphometry data (cortical thickness) from native space on the white surface of a subject. The view shows the data in tiles from 8 different angles. * **Subfigure B** illustrates arbitrary data (p-values in this case) visualized on the regions of the Desikan atlas, using the surface of the fsaverage (standard space template) subject from FreeSurfer. The view shows the data in tiles from 4 different angles. * **Subfigure C** displays the regions of the Desikan atlas on the white surface of a subject. The colors were loaded from the respective annotation file. The view shows the data in tiles from 4 different angles. *What* is displayed (morphometry data, atlas regions, arbitrary other data), on *which surface* it is displayed, and *how* it is displayed (a single interactive view, 4 tiles, 9 tiles) is independent and can be selected as needed in fsbrain. Here is a second figure, showing the same data (the [mean curvature](https://en.wikipedia.org/wiki/Mean_curvature) at each vertex) displayed on 3 different surfaces of a subject: **A** white surface, **B** pial surface, **C** inflated surface. ![Vissurfaces](./web/fsbrain_curvature_surfaces.jpg?raw=true \"Curvature visualization on different surfaces, rendered with fsbrain\") The next figure illustrates some options to visualize your results with different backgrounds. **A** Clusters on the white fsaverage surface with sulc background. **B** Region-wise p-values with curv background, inflated fsaverage surface. **C** A background color layer displaying outlines of aparc atlas regions in the respective colors, inflated demo subject surface. ![Visres](./web/fsbrain_vis_bg.jpg?raw=true \"Visualization of results and background layers, rendered with fsbrain\") ### Animations and videos Want to see brains spin? [Check this out.](./web/fsbrain_movies.md) (WARNING: loads 8 MB webpage with animated gif). ### Volume visualization Volume visualization is not the main goal of fsbrain, but standard lightbox views and simple 3D views are supported. Have a look at the vignettes or the documentation for the `volvis.lb` function. You can find some [example output here](./web/fsbrain_volume.md). ### Example Notebooks To see a combination of example figures and the code used to produce them, you should have a look at the example notebooks: [getting started notebook](https://htmlpreview.github.io/?https://github.com/dfsp-spirit/fsbrain/blob/develop/web/Rmd_web_examples/examples.html) and [advanced examples notebook](https://htmlpreview.github.io/?https://github.com/dfsp-spirit/fsbrain/blob/develop/web/Rmd_web_examples/examples_adv.html). ## Contributing Please refer to [CONTRIBUTING.md](./CONTRIBUTING.md). If you have any question, suggestion or comment on fsbrain, please [open an issue](https://github.com/dfsp-spirit/fsbrain/issues). If you want to contact me via email, please use the maintainer email address listed on the [CRAN webpage for fsbrain](https://cran.r-project.org/package=fsbrain). ## Related R packages Packages similar to fsbrain: * [ggseg](https://github.com/LCBC-UiO/ggseg) by Athanasia Mowinckel and Didac Vidal-Piñeiro: Plotting of atlas-based neuroimaging data in R. * [cerebroviz](https://github.com/ethanbahl/cerebroViz) by Ethan Bahl: Data mapping tool for visualizing spatiotemporal data in the brain. Packages used by fsbrain: * [rgl](https://CRAN.R-project.org/package=rgl) by Daniel Adler, Duncan Murdoch et al.: OpenGL-based mesh renderer. * [oro.nifti](https://github.com/muschellij2/oro.nifti) by Brandon Witcher et al. : Loading and manipulation of brain volumes from NIFTI v1 files. * [freesurferformats](https://github.com/dfsp-spirit/freesurferformats) by Tim Schäfer (me): Loading and writing various neuroimaging file formats and general mesh file formats, with a focus on FreeSurfer formats. * [gifti](https://github.com/muschellij2/gifti/) and [cifti](https://github.com/muschellij2/cifti/) by John Muschelli: Read GIFTI and CIFTI format files. * [Rvcg](https://github.com/zarquon42b/Rvcg) by Stefan Schlager: Rcpp interface for the [VCG Library](http://vcg.isti.cnr.it/vcglib/). ## Author fsbrain was written by [Tim Schäfer](https://ts.rcmd.org) Package: fsbrain Type: Package Title: Managing and Visualizing Brain Surface Data Version: 0.5.5 Authors@R: person(\"Tim\", \"Schäfer\", role = c(\"aut\", \"cre\"), email = \"ts+code@rcmd.org\", comment = c(ORCID = \"0000-0002-3683-8070\")) Maintainer: Tim Schäfer <ts+code@rcmd.org> Description: Provides high-level access to neuroimaging data from standard software packages like 'FreeSurfer' <http://freesurfer.net/> on the level of subjects and groups. Load morphometry data, surfaces and brain parcellations based on atlases. Mask data using labels, load data for specific atlas regions only, and visualize data and statistical results directly in 'R'. License: MIT + file LICENSE Encoding: UTF-8 URL: https://github.com/dfsp-spirit/fsbrain BugReports: https://github.com/dfsp-spirit/fsbrain/issues Imports: reshape, freesurferformats (>= 0.1.17), pkgfilecache (>= 0.1.1), rgl, squash, fields, viridis, data.table, magick, methods Suggests: knitr, rmarkdown, testthat (>= 2.1.0), sphereplot (>= 1.5), misc3d, RColorBrewer, Rvcg (>= 0.20.2), igraph, pracma VignetteBuilder: knitr RoxygenNote: 7.3.2 YEAR: 2019, 2020, 2021, 2022, 2023, 2024 COPYRIGHT HOLDER: Tim Schäfer cff-version: 1.2.0 message: If you use fsbrain in your research, please cite it using these metadata. title: fsbrain abstract: fsbrain is an R package for the visualization of neuroimaging data. The package can be used to visualize vertex-wise and region-wise morphometry data, parcellations, labels and statistical results on brain surfaces in three dimensions (3D). Voxel data can be displayed in lightbox mode. The fsbrain package offers various customization options and produces publication quality plots which can be displayed interactively, saved as bitmap images, or integrated into R notebooks. authors: - family-names: Schaefer given-names: Tim orcid: \"https://orcid.org/0000-0002-3683-8070\" - name: \"fsbrain: an R package for the visualization of structural neuroimaging data\" version: 0.5.3 date-released: \"2022-02-13\" identifiers: - description: All-versions DOI for fsbrain. type: doi value: \"10.5281/zenodo.3559816\" - description: Archived snapshot of version 0.5.3 of fsbrain. type: doi value: \"10.5281/zenodo.6061167\" license: MIT repository-code: \"https://github.com/dfsp-spirit/fsbrain\"",
    "stream": False
}


try:
    response = requests.post(ollama_url, json=payload)
    response.raise_for_status()

    response_data = response.json()

    if 'response' in response_data:
        print("\nExtracted Metadata:")
        print(response_data['response'])

except requests.exceptions.RequestException as e:
    print(f"Error making request to Ollama: {e}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON response from Ollama: {e}")

### 9. Flask API and ngrok Deployment

Create a Flask API to expose the model service and make it accessible externally via ngrok tunnel.

In [ ]:
!pip install pyngrok Flask

import requests
import json
import time
import threading
from pyngrok import ngrok
from flask import Flask, request, jsonify

time.sleep(15)

app = Flask(__name__)

@app.route('/generate_ollama_response', methods=['POST'])
def generate_ollama_response():
    if not request.is_json:
        return jsonify({"error": "Request must be JSON"}), 400

    data = request.get_json()
    frontend_prompt = data.get('prompt')
    model_name = data.get('model', 'metadata-extractor')

    if not frontend_prompt:
        return jsonify({"error": "Prompt not provided in request body"}), 400

    ollama_internal_url = "http://localhost:11434/api/generate"

    ollama_payload = {
        "model": model_name,
        "prompt": frontend_prompt,
        "stream": False
    }

    print(f"Forwarding request to Ollama for model '{model_name}' with prompt: {frontend_prompt[:50]}...")

    try:
        ollama_response = requests.post(ollama_internal_url, json=ollama_payload)
        ollama_response.raise_for_status()

        response_data = ollama_response.json()
        print("Received response from Ollama.")
        return jsonify(response_data)

    except requests.exceptions.ConnectionError as e:
        print(f"Connection Error to Ollama: {e}")
        return jsonify({"error": "Could not connect to Ollama. Is it running?"}), 503
    except requests.exceptions.Timeout as e:
        print(f"Timeout Error with Ollama: {e}")
        return jsonify({"error": "Ollama request timed out."}), 504
    except requests.exceptions.RequestException as e:
        print(f"Error communicating with Ollama: {e}")
        return jsonify({"error": f"Error from Ollama service: {e}"}), 500
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON response from Ollama: {e}")
        return jsonify({"error": f"Invalid JSON response from Ollama: {e}"}), 500

def run_flask_app():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

print("Starting Flask app in background thread...")
flask_thread = threading.Thread(target=run_flask_app)
flask_thread.daemon = True
flask_thread.start()

time.sleep(5)
print("Flask app should be running on port 5000.")

print("Creating ngrok tunnel for Flask app...")
try:
    ngrok.kill()
    ngrok.set_auth_token("YOUR_NGROK_AUTH_TOKEN_HERE")  # Replace with your ngrok auth token
    public_flask_url = ngrok.connect(5000).public_url
    print(f"Ngrok tunnel established! Your frontend should send requests to: {public_flask_url}/generate_ollama_response")

except Exception as e:
    print(f"Error establishing ngrok tunnel: {e}")
    print("Please ensure your ngrok authtoken is set if you have one, or check ngrok logs.")

print("\nTunnel is active. Keep this cell running to maintain the connection.")
print("You can now make POST requests from your local frontend to:")
print(f"  {public_flask_url}/generate_ollama_response")
print("Send request from frontend with a JSON body like: {'prompt': 'Repository content to analyze', 'model': 'this is the model name you have given while building the model file'}")

## 10. Integration with Frontend

The API endpoint created above serves as the backend for the Next.js frontend application. The frontend sends repository content to this endpoint for metadata extraction.

To connect the frontend with this service:

1. Copy the ngrok URL displayed in the output above
2. Add it to the frontend's `.env` file as `LLM_API_URL`
3. Set the `LLM_MODEL_NAME` to `metadata-extractor`

The frontend will then be able to communicate with this LLM service to extract metadata from repositories.